# Zero to Hero

## 1.12 Funcion de Ganancia de Arbol training/testing

El objetivo es poner el notebook anterior dentro de una funcion, e invocarla con distintas semillas, para maravillarnos ante la dispersion de las ganancias

In [1]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("/home/tomas/workspace/uba/dmeyf")  #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,612258,32.7,1327444,70.9,1124840,60.1
Vcells,1146077,8.8,8388608,64.0,1801044,13.8


Loading required package: ggplot2

Loading required package: lattice



In [2]:
GananciaArbol  <-  function( semilla, data, x, train=0.70) {
  #establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
  
  modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                          data= data[ train_rows],  #los datos de training
                          xval= 0,
                          control= x)
    
  #Aplico el modelo a los datos de testing  
  prediccion  <- predict( modelo, data[ -train_rows], type = "prob")
  
  prob_baja2  <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[ -train_rows, sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 78000, -2000) )]
  
  return( ganancia_testing)
}

Vale la pena notar que dentro de la funcion no se han creado  dtrain y dtest,  sino que directamente se utilizan:
* data[ train_rows]  como training
* data[ -train_rows]  como testing


#Aqui empieza el programa

In [3]:
dataset <- fread("./datasets/competencia1_2022.csv")   #cargo el dataset #cargo el dataset
dataset <- dataset[ foto_mes==202101 ]  # me quedo solo con el periodo 202101

In [4]:
#defino unos buenos hiperparametros
param  <- list("cp"= -0.5,
               "minsplit"=  900,
               "minbucket"= 440,
               "maxdepth"= 5
              )

In [5]:
#Ahora hago algunas llamadas a la funcion

In [6]:
GananciaArbol(  11, dataset, x=param)

[1] 6802000

In [7]:
GananciaArbol(  13, dataset, x=param)

[1] 4618000

In [8]:
GananciaArbol(  17, dataset, x=param)

[1] 4932000

In [9]:
GananciaArbol(  19, dataset, x=param)

[1] 6070000

In [10]:
GananciaArbol(  23, dataset, x=param)

[1] 6332000

Lo primero que se obseva es la gran variabilidad de la ganancia sin normalizar segun la semilla, hasta un 50%

| semilla | ganancia |
| --- | ---: |
| 11 | 6802000 |
| 13 | 4618000 |
| 17 | 4932000 |
| 19 | 6070000 |
| 23 | 6332000 |



Normalizando las ganancias de testing, es decir haciendo el cociente por 0.3 da lo siguiente

| semilla | ganancia |
| --- | ---: |
| 11 | 22673333|
| 13 | 15393333 |
| 17 | 16440000 |
| 19 | 20233333 |
| 23 | 21106667 |

Esta dispersión es **NOTABLE**  ya que el algoritmo es el mismo, con los mismos parámetros.
<br>
Lo único que cambia es que datos se utilizan para entrenar y testear, pero siempre es 70% / 30%
<br>
Es más notable aún que se ha tenido cuidado que la partición sea estratificada segun el campo **clase_ternaria**, lo que apriori uno supondria que va a generar particiones muy homogéneas.